# 本代码把早年晚年的时间序列给挑出来，每年的季风爆发前20天内求个平均

In [1]:
import xarray as xr
import numpy as np
import os
import math
import sys
module_path = ["/home/sun/mycode/module/","/data5/2019swh/mycode/module/"]
sys.path.append(module_path[0])
from module_sun import *

np.set_printoptions(suppress=True)

In [2]:
#获取时间
with open("/home/sun/qomo-data/early_date.json",'r') as load_f:
    a = json.load(load_f)
early_years = np.array(list(a.keys()))
early_days = np.array(list(a.values()))
early_years = early_years.astype(np.int)
early_days = early_days.astype(np.int)

with open("/home/sun/qomo-data/late_date.json",'r') as load_f:
    a = json.load(load_f)
late_years = np.array(list(a.keys()))
late_days = np.array(list(a.values()))
late_years = late_years.astype(np.int)
late_days = late_days.astype(np.int)

/tmp/ipykernel_183517/1700115208.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  early_years = early_years.astype(np.int)
/tmp/ipykernel_183517/1700115208.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in Num

In [3]:
print(np.mean(late_days))
print(out_date(1981,134))

134.33333333333334
0514


In [4]:
test  =  xr.open_dataset("/home/sun/qomo-data/burst_seris/1980_composite_uwind.nc")
testv  =  xr.open_dataset("/home/sun/qomo-data/burst_seris/1980_composite_vwind.nc")
path  =  "/home/sun/qomo-data/burst_seris/"
#声明存储的数组
early_u  =  np.zeros((len(early_years),test.uwind.data.shape[1],test.uwind.data.shape[2],test.uwind.data.shape[3]))  #早年的年份数、层次、维度、经度
early_v  =  early_u.copy()
late_u   =  np.zeros((len(late_years),test.uwind.data.shape[1],test.uwind.data.shape[2],test.uwind.data.shape[3]))  #晚年的年份数、层次、维度、经度
late_v   =  late_u.copy()

for i in range(0,len(early_years)):
    f_u  =  xr.open_dataset(path+str(early_years[i])+"_composite_uwind.nc")
    f_v  =  xr.open_dataset(path+str(early_years[i])+"_composite_vwind.nc")

    early_u[i,:]  =  np.average(f_u.uwind.data[0:20,:],axis=0)
    early_v[i,:]  =  np.average(f_v.vwind.data[0:20,:],axis=0)

for i in range(0,len(late_years)):
    f_u  =  xr.open_dataset(path+str(late_years[i])+"_composite_uwind.nc")
    f_v  =  xr.open_dataset(path+str(late_years[i])+"_composite_vwind.nc")

    late_u[i,:]  =  np.average(f_u.uwind.data[0:20,:],axis=0)
    late_v[i,:]  =  np.average(f_v.vwind.data[0:20,:],axis=0)

In [5]:
ncfile  =  xr.Dataset(
    {
        "early_u": (["early_series", "lev", "lat", "lon"], early_u),
        "early_v": (["early_series", "lev", "lat", "lon"], early_v),
        "late_u":  (["late_series", "lev", "lat", "lon"], late_u),
        "late_v":  (["late_series", "lev", "lat", "lon"], late_v),
    },
    coords={
        "lon": (["lon"], test.lon.data),
        "lat": (["lat"], test.lat.data),
        "lev": (["lev"], test.level.data),
        "early_series": (["early_series"], np.linspace(1,len(early_years),len(early_years))),
        "late_series":  (["late_series"],  np.linspace(1,len(late_years),len(late_years))),
    },
)
ncfile.early_u.attrs  =  test.uwind.attrs
ncfile.early_v.attrs  =  testv.vwind.attrs
ncfile.late_u.attrs   =  test.uwind.attrs
ncfile.late_v.attrs   =  testv.vwind.attrs
ncfile["lat"].attrs  =  test["lat"].attrs
ncfile["lon"].attrs  =  test["lon"].attrs
ncfile["lev"].attrs  =  test["level"].attrs

ncfile.attrs["description"]  =  "In this file,I computed 20days average before monsoon onset,D0-30~D0-10.One time means this years average"
ncfile.attrs["created time"] =  "2021-10-22"
ncfile.to_netcdf("/home/sun/data/early_late_year_average_before_onset_wind.nc")